# Regularyzacja LASSO, Ridge, Elastic Net

## Przedstawienie problemu

Ponizsze dane zawierają informację na temat współczynnika genetycznego pewnej choroby oraz ekspresji genów dla $1000$ pacjentów. Naszym zadaniem będzie sprawdzenie zależności (zdudowanie modelu statystycznego) pomiędzy informacją genetyczną, a indeksem chorobowym. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

gene_data = pd.read_csv("data/gene expression.csv")
gene_data

Poniewaz zmienna `disease_indicator` przyjmuje wartości z przestrzeni liczb rzeczywistych moglibyśmy użyć znanego nam już modelu regresji liniowej.

In [ ]:
# Zadanie 1
# Podziel zbiór na treningowy i testowy
# Zbuduj model regresji liniowej
# Podaj błąd kwadratowy na zbiorze treningowym i testowym
# Podaj R^2 modelu

Od razu widzimy, że coś jest nie tak. Na zbiorze treningowym osiaga prawie zerowy błąd, na zbiorze testowym jednak wypada on fatalnie. Jest to typowy przypkad **overfittingu**, czyli **zbyt wysokiej wariancji modelu**. Overfitting spowodowany jest uzyciem zbyt elastycznego modelu, jednakże zazwyczaj rozumiemy to poprzez użycie zbyt elastycznej architektury jak np. sieci nauronowego do prostego problemu. Model regresji liniowej jest bardzo prostym modelem więc nie powinno byc z tym problemu.

Overfitting może jednak występować także dla prostych modeli w przypadku gdy posiadamy więcej predyktorów niż obserwacji: $p \geq N$. We wspomianym przypadku estymatory oparte na metodzie największej wiarygodności jak i na metodzie najmniejszych kwadratów narażone sa na przeuczenie modelu. 

Zwizualizujmy co się dzieje na prostym przykładzie zakładając, że mamy w naszym zbiorze treningowym tylko 2 obserwacje i używamy jednego predyktora (dodając wyraz wolny mamy $p=N$). W takim przypadku łatwo zauważyć, że model regresji liniowej da nam po prostu wzór na linię przechodzącą przez obie obserwacje:

In [ ]:
np.random.seed(2222)

sample_data = pd.DataFrame({
    "x": [1, 2, 3, 4, 5, 6, 7],
    "y": [2*x + 1 + np.random.normal() for x in range(1, 8)],
    "set": ["train", "test", "train", "test", "test", "test", "test"]
})

train_data = sample_data.loc[sample_data["set"] == "train", ["x", "y"]]

test_model = LinearRegression().fit(train_data[["x"]], train_data["y"])
test_model_intercept = test_model.intercept_
test_model_slope = test_model.coef_[0]

sns.scatterplot(data=sample_data, x="x", y="y", hue="set") 
plt.plot(sample_data["x"], test_model_intercept + test_model_slope * sample_data["x"], color='black')
plt.show()

## Regresja grzbietowa

Aby rozwiazać ten problem musimy zastanowić się jak zmnieszyć wariancję naszego modelu. Wariancja i obciążenie (bias) są ze sobą ścisle powiązane. Zmniejszająć jedno zwiększamy drugie - bias-variance tradeoff, tak więc moglibyśmy sprawdzić co stanie się gdy do naszego modelu dodamy obciażanie. dokonamy tego dodając **regularyzacji** naszego modelu, karając model za zbyt wysokie współczynniki $\beta$. Mówiąc dokładniej w regularyzowanej wersji naszego modelu minimalizować będziemy nie loglikelihood (lub błąd kwadratowy), a poniższe wyrażenie:

$$
\hat{\beta} = agrmin_{\beta} (-\ell(\beta) + \lambda\sum_{i=1}^p \beta_p^2)
$$

Jest to tak zwana **regresja grzbietowa** (Ridge regression). Parametr $\lambda$ jest **hiperparametrem** modelu, który musimy ustalić. Jest to tak zwany **współczynnik kary** - im większa $\lambda$ tym bardziej "karamy" nasz model za zbyt duże wartości parametrów. W przypadku regresji liniowej wartość likelihood mozemy zastąpić błędem kwadratowym, i bezpośrednio znaleźć postać rozwiązania:

$$
\hat{\beta} = (X^TX + \lambda I)^{-1}X^Ty
$$
Zwizualizujmy rozwiązanie regresji grzbietowej dla kilku wartości $\lambda$:

In [ ]:
X = np.c_[np.ones(len(sample_data[sample_data["set"] == "train"])), 
          sample_data[sample_data["set"] == "train"][["x"]].values]
y = sample_data[sample_data["set"] == "train"][["y"]].values

ridge_intercept = np.mean(y)

solution_lambda_0 = np.linalg.inv(X.T @ X) @ X.T @ y
solution_lambda_1 = np.linalg.inv(X.T @ X + 1*np.identity(X.shape[1])) @ X.T @ y
solution_lambda_5 = np.linalg.inv(X.T @ X + 5*np.identity(X.shape[1])) @ X.T @ y
solution_lambda_10 = np.linalg.inv(X.T @ X + 10*np.identity(X.shape[1])) @ X.T @ y

sns.scatterplot(data=sample_data, x="x", y="y", hue="set") 
plt.plot(sample_data["x"], 
         test_model_intercept + test_model_slope * sample_data["x"], 
         color='blue')
plt.text(1.5, 14, "lambda=0", color="blue")
plt.plot(sample_data["x"], 
         solution_lambda_1[0,0] + solution_lambda_1[1,0] * sample_data["x"], 
         color='green')
plt.text(1.5, 13, "lambda=1", color="green")
plt.plot(sample_data["x"], 
         solution_lambda_5[0,0] + solution_lambda_5[1,0] * sample_data["x"], 
         color='red')
plt.text(1.5, 12, "lambda=5", color="red")
plt.plot(sample_data["x"], 
         solution_lambda_10[0,0] + solution_lambda_10[1,0] * sample_data["x"], 
         color='black')
plt.text(1.5, 11, "lambda=10", color="black")
plt.show()

Oczywiście pojawia się pytanie, jak wybrać wielkość $lambda$ ? Parametr ten wybierany jest w procesie **cross-walidacji** (CV). CV jest prostą lecz skuteczną metodą estymacji błędu generalizacji dlatego też doskonale nadaje się do estymacji wartości wszelkich hiperparametrów. 

W procesie CV dzielimy nasz zbiór treningowy na **foldy** równej wielkości. Nastepnie budujemy $K$ modeli za nowy zbiór treningowy biorąc $K-1$ foldów - w kazdym z modeli pozostający fold jest traktowany jako zbiór walidacyjny. Nastepnie uśredniamy wyniki (np. bład sredniokwadratowy) ze zbiorów walidacyjnych otrzymując estymację zbioru generalizacyjnego.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

X = gene_data.drop('disease_indicator', axis=1)
y = gene_data['disease_indicator']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

param_grid = {'alpha': np.logspace(-4, 4, 9)}

ridge = Ridge()
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=10)

grid_search.fit(X, y)
best_params = grid_search.best_params_
coefs = []
mse = []
for a in param_grid['alpha']:
    ridge = Ridge(alpha=a)
    ridge.fit(X, y)
    coefs.append(ridge.coef_)
    mse.append(np.mean((ridge.predict(X) - y) ** 2))

print(f'Najlepsza wartość lambda: {best_params["alpha"]}')
print(f'Wynik regresji: {grid_search.best_score_}')

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(param_grid['alpha'], mse)
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('MSE')
plt.title('MSE w zależności od Lambda')
plt.axis('tight')

plt.subplot(1, 2, 2)
plt.plot(param_grid['alpha'], coefs)
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('Współczynniki')
plt.title('Współczynniki Ridge w zależności od Lambda')
plt.axis('tight')

plt.show()

Mozy teraz policzyć predykcje:

In [ ]:
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
print("Błąd kwadratowy dla zbioru treningowego: ", mse_train)
print("Błąd kwadratowy dla zbioru testowego: ", mse_test)

Regresja grzbietowa jest przykładem **regularyzacji normą $L^q$** gdzie $q=2$:

$$
\hat{\beta} = agrmin_{\beta} (-\ell(\beta) + \lambda\sum_{i=1}^p |\beta_p|^q)
$$

In [ ]:
def gen_norm(x, q):
    return (1 - abs(x ** q)) ** (1/q)

q_norms = pd.DataFrame()
q_values = [0.3, 0.5, 1, 2, 3, 5, 20]

for q in q_values:
    points = pd.DataFrame({
        'x': np.arange(0, 1.001, 0.001),
        'y': gen_norm(np.arange(0, 1.001, 0.001), q),
        'q': q
    })
    points_1 = points.copy()
    points_2 = points.copy()
    points_3 = points.copy()
    points_4 = points.copy()
    points_1['y'] = -points_1['y']
    points_2['x'] = -points_2['x']
    points_3['x'] = -points_3['x']
    points_3['y'] = -points_3['y']
    q_norms = pd.concat([q_norms, points, points_1, points_2, points_3, points_4])

q_norms['q'] = q_norms['q'].astype('category')
q_norms = q_norms.sort_values(['q', 'y', 'x'])

plt.figure(figsize=(8, 6))
plt.scatter(q_norms['x'], q_norms['y'], c=q_norms['q'], cmap='viridis', alpha=0.8, s=5)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Kula w przestrzenie L^q')
plt.show()

## Regresja LASSO

Moznaby się zastanowić co stanie się gdy użyjemy innej normy do regularyzacji. Najczęsciej spotykanym kuzynem regresji grzbietowej jest **regresja LASSO** (least absolute shrinkage and selection operator), gdzie $q=1$:

$$
\hat{\beta} = agrmin_{\beta} (-\ell(\beta) + \lambda\sum_{i=1}^p |\beta_p|)
$$


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

X = gene_data.drop('disease_indicator', axis=1)
y = gene_data['disease_indicator']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

param_grid = {'alpha': np.logspace(-4, 4, 9)}

lasso = Lasso()
grid_search = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=10)

grid_search.fit(X, y)
best_params = grid_search.best_params_
coefs = []
mse = []
for a in param_grid['alpha']:
    lasso = Lasso(alpha=a)
    lasso.fit(X, y)
    coefs.append(lasso.coef_)
    mse.append(np.mean((lasso.predict(X) - y) ** 2))

print(f'Najlepsza wartość lambda: {best_params["alpha"]}')
print(f'Wynik regresji: {grid_search.best_score_}')

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(param_grid['alpha'], mse)
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('MSE')
plt.title('MSE w zależności od Lambda')
plt.axis('tight')

plt.subplot(1, 2, 2)
plt.plot(param_grid['alpha'], coefs)
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('Współczynniki')
plt.title('Współczynniki Ridge w zależności od Lambda')
plt.axis('tight')

plt.show()

In [ ]:
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
print("Błąd kwadratowy dla zbioru treningowego: ", mse_train)
print("Błąd kwadratowy dla zbioru testowego: ", mse_test)

Widać tu pewną różnicę jesli chodzi o wartości współczynników $\beta$. W regresji grzbietowej niektóre wartości są bliskie 0, ale nigdy nie osiągają tej wartości - metoda regresji grzbietowej pozwala tylko na dojście asymptotycznie blisko do 0. W przypadku LASSO dostajemy zerowe współczynniki - daje nam to bardzo przydatną własność, bo LASSO działa jak **selektor zmiennych**.

## Regresja Elastic Net

LASSO daje z reguły lepsze wyniki gdy w naszym zbiorze mamy nieistotne predyktory (zmienne które nie wnoszą informacji do modelu), LASSO potrafi się ich pozbyć. Regresja grzbietowa działa lepiej gdy predyktory są istotne. Jeśli chcemy korzystać z zalet obu podejść możemy użyć regresji **Elastic Net**:

$$
\hat{\beta} = agrmin_{\beta} (-\ell(\beta) + \lambda(\alpha\sum_{i=1}^p |\beta_p| + (1-\alpha)\sum_{i=1}^p \beta_p^2))
$$

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

X = gene_data.drop('disease_indicator', axis=1)
y = gene_data['disease_indicator']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

param_grid = {'alpha': np.logspace(-4, 4, 9)}

elastic_net = ElasticNet(l1_ratio=0.5)
grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=10)

grid_search.fit(X, y)
best_params = grid_search.best_params_
coefs = []
mse = []
for a in param_grid['alpha']:
    elastic_net = ElasticNet(alpha=a, l1_ratio=0.5)
    elastic_net.fit(X, y)
    coefs.append(elastic_net.coef_)
    mse.append(np.mean((elastic_net.predict(X) - y) ** 2))

print(f'Najlepsza wartość lambda: {best_params["alpha"]}')
print(f'Wynik regresji: {grid_search.best_score_}')

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(param_grid['alpha'], mse)
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('MSE')
plt.title('MSE w zależności od Lambda')
plt.axis('tight')

plt.subplot(1, 2, 2)
plt.plot(param_grid['alpha'], coefs)
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('Współczynniki')
plt.title('Współczynniki Ridge w zależności od Lambda')
plt.axis('tight')

plt.show()

In [ ]:
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
print("Błąd kwadratowy dla zbioru treningowego: ", mse_train)
print("Błąd kwadratowy dla zbioru testowego: ", mse_test)

Wartość $\alpha$ jest kolejnym hiperparametrem który należałoby wybrać z użyciem CV i metody seleckji jak np grid search.

# Zadanie
Zbuduj model regresji logistycznej z regularyzacją dla poniższego zbioru danych.
Sprawdź braki danych, rozkłady zmiennych, użyj modułu scikit-learn.

In [ ]:
import pickle
import os
import numpy as np

filepaths = os.listdir("data/creditcard/")
for fp in filepaths:
    with open("data/creditcard/" + fp, 'rb') as f:
        globals()[fp.replace(".pickle", "")] = pickle.load(f)